## CommaSeparatedListOutputParser
* 쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용
* 이 방법은 정보를 구조화하고, 가독성을 높이며, 특히 데이터를 다루거나 리스트 형태의 결과를 요구하는 경우에 매우 유용

In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = prompt | model | output_parser

chain.invoke({"subjects", "대한민국 유명여가수"})

['아이유', '태연', '제니', '화사', '청하']

## StructuredOutputParser
* 여러 필드를 반환하고자 할 때 사용할 수 있습니다. Pydantic/JSON 파서가 더 강력하지만, 이는 덜 강력한 모델에 유용

In [7]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 출처, 웹사이트 이여야 합니다",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the user question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)
model = ChatOpenAI(model_name="gpt-4o", temperature=0)
chain = prompt | model | output_parser

chain.invoke({"question": "sum41 그룹에 대해서 알려주세요."})

{'answer': "Sum 41은 캐나다의 펑크 록 밴드로, 1996년에 결성되었습니다. 밴드는 보컬리스트이자 기타리스트인 Deryck Whibley, 기타리스트 Dave Baksh, 베이시스트 Jason 'Cone' McCaslin, 드러머 Frank Zummo로 구성되어 있습니다. 그들의 음악은 펑크 록, 팝 펑크, 얼터너티브 록 등의 장르를 포함하며, 대표적인 앨범으로는 'All Killer No Filler', 'Does This Look Infected?', 'Chuck' 등이 있습니다.",
 'source': 'https://en.wikipedia.org/wiki/Sum_41'}

## JsonOutputParser
* 사용자가 원하는 JSON 스키마를 지정
* 모델의 용량이 충분해야 함

In [13]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Topic(BaseModel):
    description: str = Field(description="주제에 대한 설명")
    hashtags: List[str] = Field(description="해시태그 형태의 키워드")

query = "offspring 그룹에 대해서 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)

prompt = PromptTemplate(
    template="answer the user question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"question": query})

{'description': "The Offspring는 1984년에 결성된 미국의 록 밴드입니다. 이 밴드는 보컬리스트 Dexter Holland, 기타리스트 Noodles, 베이시스트 Todd Morse, 드러머 Pete Parada로 구성되어 있습니다. The Offspring는 1994년에 발매된 앨범 'Smash'로 큰 인기를 얻었으며, 이 앨범은 독립 레이블에서 발매된 앨범 중 가장 많이 팔린 앨범 중 하나입니다.",
 'hashtags': ['TheOffspring',
  'RockBand',
  'SmashAlbum',
  'DexterHolland',
  'Noodles',
  'ToddMorse',
  'PeteParada']}

In [14]:
# without pydantic
query = "온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."
parser = JsonOutputParser()
prompt = PromptTemplate(
    template="answer the user question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | model | parser
chain.invoke({"question": query})

{'description': '온난화는 지구의 평균 기온이 장기적으로 상승하는 현상을 말합니다. 이는 주로 인간 활동에 의해 발생하는 온실가스 배출 증가로 인해 발생합니다. 온난화는 기후 변화, 해수면 상승, 극지방 빙하 감소, 생태계 변화 등 다양한 환경 문제를 초래합니다.',
 'hashtags': ['#온난화', '#기후변화', '#온실가스', '#지구온난화', '#환경문제', '#해수면상승', '#빙하감소']}

## PandasDataFrameOutputParser
* 구조화된 데이터를 다루기 위한 포괄적인 도구 세트를 제공하여, 데이터 정제, 변환 및 분석과 같은 작업에 다양하게 활용
* 사용자가 임의의 Pandas DataFrame을 지정하고 해당 DataFrame에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있는 LLM을 요청할 수 있게 해줌

In [29]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import (PandasDataFrameOutputParser)

def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

df = pd.read_csv("../../data/titanic.csv")
df.head()
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [30]:
parser = PandasDataFrameOutputParser(dataframe=df)
df_query = "retrieve the psddrngers age column"

prompt = PromptTemplate(
    template="answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})
format_parser_output(parser_output)

OutputParserException: Unsupported request type '"column'.                         Please check the format instructions.